In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from trees import Tree, kNT, TPT
import numpy as np

In [2]:
def get_canonical_label(tree, node, subtree_dict, current_label_dict):
    '''Get the canonical label for the subtree rooted at the given node.'''
    tree.sort_tree(node)
    children = tree.chi(node)
    if not children:
        subtree_form = (tree.node_labels[node]['name'],)
    else:
        subtree_form = (tree.node_labels[node]['name'], tuple(get_canonical_label(tree, child, subtree_dict, current_label_dict) for child in children))

    if subtree_form not in subtree_dict:
        subtree_dict[subtree_form] = current_label_dict['current_label']
        current_label_dict['current_label'] += 1

    return subtree_dict[subtree_form]

def canonize_tree(tree, subtree_dict, current_label_dict):
    '''Canonize the tree and return the canonical form of the root node and the dictionary of subtrees.'''
    canonical_form = get_canonical_label(tree, tree.root, subtree_dict, current_label_dict)
    return canonical_form, subtree_dict

In [3]:
def read_g6_file(file_path):
    # Read the graph from a g6 format file
    graphs = list(nx.read_graph6(file_path))
    
    # Convert node labels to strings
    for i in range(len(graphs)):
        graphs[i] = nx.relabel_nodes(graphs[i], str)
    
    return graphs

In [4]:
def check_distinctness(file_path):
    Gs = read_g6_file(file_path)
    k = 1
    shared_subtree_dict = {}
    current_label_dict = {'current_label': 0}

    knt_forest = []
    tpt_forest = []
    for G in Gs:
        knt_canons = []
        tpt_canons = []
        for node in G.nodes:
            knt_instance = kNT(G=G, root=node, k=k)
            for knt_node in knt_instance.unfolding_tree.nodes():
                # Treat all nodes as having the same label '0'
                knt_instance.node_labels[knt_node] = {'name': '0'}
            tpt_instance = TPT(G=G, root=node)
            for tpt_node in tpt_instance.tpt_tree.nodes():
                tpt_instance.node_labels[node] = {'name': '0'}

            knt_canon, shared_subtree_dict = canonize_tree(knt_instance, shared_subtree_dict, current_label_dict)
            tpt_canon, shared_subtree_dict = canonize_tree(tpt_instance, shared_subtree_dict, current_label_dict)

            knt_canons.append(knt_canon)
            tpt_canons.append(tpt_canon)
        
        knt_forest.append(knt_canons)
        tpt_forest.append(tpt_canons)

    # Convert the forests to numpy arrays if needed
    knt_forest = np.array(knt_forest, dtype=object)
    tpt_forest = np.array(tpt_forest, dtype=object)

    # print(knt_forest)
    # print(tpt_forest)

    # Function to compare two lists of strings
    def compare_lists(list1, list2):
        return all(x == y for x, y in zip(list1, list2)) and len(list1) == len(list2)

    # Function to check all combinations in a forest
    def check_forest(forest):
        n = len(forest)
        for i in range(n):
            for j in range(i + 1, n):
                if compare_lists(forest[i], forest[j]):
                    return True, i, j
        return False, None, None

    # Check knt_forest
    knt_has_duplicates, knt_i, knt_j = check_forest(knt_forest)

    # Check tpt_forest
    tpt_has_duplicates, tpt_i, tpt_j = check_forest(tpt_forest)

    # Print results
    if knt_has_duplicates:
        print(f"knt_forest has duplicates: knt_forest[{knt_i}] is equal to knt_forest[{knt_j}]")
    else:
        print("knt_forest: All elements are distinct")

    if tpt_has_duplicates:
        print(f"tpt_forest has duplicates: tpt_forest[{tpt_i}] is equal to tpt_forest[{tpt_j}]")
    else:
        print("tpt_forest: All elements are distinct")

## Test on strongly regular graphs

In [7]:
check_distinctness('strongly_regular_graphs/sr251256.g6')

KeyboardInterrupt: 

## Test on simple graphs

In [44]:
check_distinctness('simple_graphs/graph2.g6')
check_distinctness('simple_graphs/graph3.g6')
check_distinctness('simple_graphs/graph4.g6')
check_distinctness('simple_graphs/graph5.g6')
check_distinctness('simple_graphs/graph6.g6')
check_distinctness('simple_graphs/graph7.g6')
# check_distinctness('simple_graphs/graph8.g6')
# check_distinctness('simple_graphs/graph9.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on simple connected graphs

In [45]:
# check_distinctness('simple_graphs/graph2.g6')
check_distinctness('simple_graphs/graph3c.g6')
check_distinctness('simple_graphs/graph4c.g6')
check_distinctness('simple_graphs/graph5c.g6')
check_distinctness('simple_graphs/graph6c.g6')
check_distinctness('simple_graphs/graph7c.g6')
# check_distinctness('simple_graphs/graph8.g6')
# check_distinctness('simple_graphs/graph9.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on Eulerian graphs

In [52]:
check_distinctness('eulerian_graphs/eul3.g6')
check_distinctness('eulerian_graphs/eul4.g6')
check_distinctness('eulerian_graphs/eul5.g6')
check_distinctness('eulerian_graphs/eul6.g6')
check_distinctness('eulerian_graphs/eul7.g6')
check_distinctness('eulerian_graphs/eul8.g6')
# check_distinctness('eulerian_graphs/eul9.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on connected Eulerian graphs

In [53]:
# check_distinctness('eulerian_graphs/eul3c.g6')
# check_distinctness('eulerian_graphs/eul4c.g6')
check_distinctness('eulerian_graphs/eul5c.g6')
check_distinctness('eulerian_graphs/eul6c.g6')
check_distinctness('eulerian_graphs/eul7c.g6')
check_distinctness('eulerian_graphs/eul8c.g6')
# check_distinctness('eulerian_graphs/eul9c.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on perfect graphs

In [56]:
check_distinctness('perfect_graphs/perfect5.g6')
check_distinctness('perfect_graphs/perfect6.g6')
check_distinctness('perfect_graphs/perfect7.g6')
# check_distinctness('perfect_graphs/perfect8.g6')
# check_distinctness('perfect_graphs/perfect9.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on Hypohamiltonian graphs

In [57]:
check_distinctness('hypo_hamiltonian_graphs/hypo16.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on Planar graphs

In [59]:
check_distinctness('planar_graphs/planar_conn.3.g6')
check_distinctness('planar_graphs/planar_conn.4.g6')
check_distinctness('planar_graphs/planar_conn.5.g6')
check_distinctness('planar_graphs/planar_conn.6.g6')
check_distinctness('planar_graphs/planar_conn.7.g6')
check_distinctness('planar_graphs/planar_conn.8.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on self-complementary graphs

In [5]:
check_distinctness('self_comp_graphs/selfcomp5.g6')
check_distinctness('self_comp_graphs/selfcomp8.g6')
check_distinctness('self_comp_graphs/selfcomp9.g6')
# check_distinctness('self_comp_graphs/selfcomp12.g6')
# check_distinctness('self_comp_graphs/selfcomp13.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct


## Test on highly irregular graphs

In [6]:
check_distinctness('highly_irregular_graphs/highlyirregular8.g6')
check_distinctness('highly_irregular_graphs/highlyirregular9.g6')
check_distinctness('highly_irregular_graphs/highlyirregular10.g6')
check_distinctness('highly_irregular_graphs/highlyirregular11.g6')
check_distinctness('highly_irregular_graphs/highlyirregular12.g6')
check_distinctness('highly_irregular_graphs/highlyirregular13.g6')
# check_distinctness('highly_irregular_graphs/highlyirregular14.g6')

knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
knt_forest: All elements are distinct
tpt_forest: All elements are distinct
